<a href="https://colab.research.google.com/github/leman-cap13/my_projects/blob/main/Multi_Task_NER_POS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
files.upload()

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download rohitr4307/ner-dataset

In [ ]:
import zipfile

In [ ]:
with zipfile.ZipFile('/content/ner-dataset.zip','r') as zip_ref:
  zip_ref.extractall()

In [ ]:
import pandas as pd
df=pd.read_csv('/content/NER_Dataset.csv')
df

In [ ]:
df.columns

In [ ]:
df['Word'].iloc[4]

In [ ]:
df['Tag'].unique()

In [ ]:
#B-gpe, B-tim, B-geo, B-org,I-geo, I-org, I-tim ---BIO Formats

In [ ]:
import numpy as np

In [ ]:
# First Step Tokenization
# our input is already token

In [ ]:
# Convert Token to token id

In [ ]:
import ast
df['Word'] = df['Word'].apply(ast.literal_eval)


In [ ]:
# Pos ucun Eval appyle et
df['POS']=df['POS'].apply(ast.literal_eval)

In [ ]:
df['Word'].head()

In [ ]:
all_tokens = [token for row in df['Word'] for token in row]
all_tokens

In [ ]:
vocab = {token: idx+1 for idx, token in enumerate(sorted(set(all_tokens)))}

In [ ]:
vocab_size=len(vocab)+1

In [ ]:
vocab_size

In [ ]:
def tokens_to_ids(token_list, vocab):
    return [vocab.get(token, 0) for token in token_list]

token_ids = tokens_to_ids(all_tokens, vocab)


In [ ]:
df['token_ids'] = df['Word'].apply(lambda tokens: tokens_to_ids(tokens, vocab))

In [ ]:
df['token_ids']

In [ ]:
# tag token id cevirmeliyik

In [ ]:
df['Tag'].dtype

In [ ]:
import ast

df['Tag'] = df['Tag'].apply(ast.literal_eval)


In [ ]:
#uniques for POS
uniques_pos=sorted(df['POS'].explode().unique())
pos2id={pos: idx for idx,pos in enumerate(uniques_pos)}
id2pos={idx: pos for pos,idx in pos2id.items()}

In [ ]:
uniques_tag=sorted(df['Tag'].explode().unique())
uniques_tag

In [ ]:
tag2id = {tag: idx for idx, tag in enumerate(uniques_tag)}
id2tag = {idx: tag for tag, idx in tag2id.items()}

In [ ]:
tag2id

In [ ]:
id2tag

In [ ]:
df['tag_ids'] = df['Tag'].apply(lambda tags: [tag2id[tag] for tag in tags])

In [ ]:
df['pos_ids']  = df['POS'].apply(lambda tags: [pos2id[tag] for tag in tags])

In [ ]:
df['tag_ids']

In [ ]:
# Indi Token id leri paddin etmek lazimdiki hem imput hem output uzunlugu eyni olsun

In [ ]:
import tensorflow as tf

In [ ]:
df['seq_len'] = df['token_ids'].apply(len)

print("Max Length:", df['seq_len'].max())

In [ ]:
MAX_LEN = int(df['seq_len'].quantile(0.95))

In [ ]:
MAX_LEN

In [ ]:
print(pos2id.keys())

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

MAX_LEN = 35

df['input_ids'] = list(pad_sequences(df['token_ids'], maxlen=MAX_LEN, padding='post', value=0))
df['label_ids'] = list(pad_sequences(df['tag_ids'], maxlen=MAX_LEN, padding='post', value=tag2id['O']))

# pos ucun padding
df['pos_label_ids']=list(pad_sequences(df['pos_ids'], maxlen=MAX_LEN, padding='post',value=pos2id['NN']))


In [ ]:
df['input_ids']

In [ ]:
df['label_ids']

In [ ]:
df['pos_label_ids']

In [ ]:
# Train teste bolek demeli hem POS ucun Hem de NER ucun ayri ayri output olmalidi multi Task ucun ona gore y hissesini y_ner ve
# y_pos kimi ayirdiqdan sonra train_test_split istifade edecem

In [ ]:
from sklearn.model_selection import train_test_split
X = np.array(df['input_ids'].to_list(),dtype=np.int32)
y_ner = np.array(df['label_ids'].to_list(),dtype=np.int32)
y_pos=np.array(df['pos_label_ids'].to_list(), dtype=np.int32)

# Pozitional ID-lər (0, 1, 2, ..., MAX_LEN-1)
pos_ids = np.tile(np.arange(X.shape[1]), (X.shape[0], 1))

X_train, X_test, y_ner_train, y_ner_test, y_pos_train, y_pos_test, pos_train, pos_test = train_test_split(
    X, y_ner, y_pos, pos_ids, test_size=0.2, random_state=42
)

In [ ]:
X.shape[0], y_ner.shape[0], y_pos.shape[0], pos_ids.shape[0]

In [ ]:
print("X dtype:", X.dtype)

In [ ]:
# Tensorflow datasina cevirek

In [ ]:
batch_size=32

train_dataset = tf.data.Dataset.from_tensor_slices(
    ((X_train, pos_train), (y_ner_train, y_pos_train))
).batch(batch_size).shuffle(1000).prefetch(tf.data.AUTOTUNE)


valid_dataset = tf.data.Dataset.from_tensor_slices(
    ((X_test, pos_test), (y_ner_test, y_pos_test))
).batch(batch_size).prefetch(tf.data.AUTOTUNE)

In [ ]:
train_dataset

In [ ]:
valid_dataset

In [ ]:
#Encoder base transformer model

encoder_inputs=tf.keras.Input(shape=[None,], name='encoder_inputs')

# encoder ama bu defe POS ucu
pos_encoder_inputs=tf.keras.Input(shape=[None,], name='pos_encoder_inputs')

#Embedding layer
embed_layer=tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=128, mask_zero=True, name='embed_layer')

# Same Pos ucun emdedding layer
pos_em_lay=tf.keras.layers.Embedding(input_dim=vocab_size,output_dim=128,mask_zero=True, name='pos_em_layer')

#encoder embedding
encoder_embed=embed_layer(encoder_inputs)
pos_encoder_embed=pos_em_lay(pos_encoder_inputs)
sum_pos_and_input_embed=encoder_embed+pos_encoder_embed


#Positional embed layer
embed_size=128
pos_embed_layer=tf.keras.layers.Embedding(MAX_LEN, embed_size)

pos_encoder=tf.keras.layers.Lambda(lambda x: tf.range(start=0,limit=tf.shape(x)[1],
                                                            delta=1))(encoder_inputs)

positional_embedding=pos_embed_layer(pos_encoder)




#concat tokrn and positional embedding
encoder_embedding=positional_embedding+sum_pos_and_input_embed

#Add multihead attentioan layer

num_heads=3
encoder_attention=tf.keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_size)(encoder_embedding,encoder_embedding)
encoder_attention=tf.keras.layers.LayerNormalization(epsilon=1e-6)(encoder_attention+encoder_embedding)


# add Feed Forward (Dense) layer
ff_dim=512
encoder_ff=tf.keras.layers.Dense(ff_dim, activation='relu')(encoder_attention)
encoder_ff=tf.keras.layers.Dense(embed_size)(encoder_ff)
encoder_ff=tf.keras.layers.LayerNormalization(epsilon=1e-6)(encoder_ff+encoder_attention)


#Add output layer Multi task olduguna gore hem Ner ucun Hem POS ucun ayri ayri output yazmaliyiq
ner_output=tf.keras.layers.Dense(len(tag2id), activation='softmax', name='ner_output')(encoder_ff)

pos_output=tf.keras.layers.Dense(len(pos2id), activation='softmax', name='pos_output')(encoder_ff)

In [ ]:
#Callback and learning schedule
learning_rate_schedule=tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-3,
    decay_steps=1000,
    decay_rate=0.9
)

early_stopping=tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

In [ ]:
#Make model
model=tf.keras.Model(inputs=[encoder_inputs,pos_encoder_inputs], outputs=[ner_output, pos_output])

In [ ]:
#optimizer
optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate_schedule)

In [ ]:
#model compiling
model.compile(optimizer=optimizer,

              loss= {'ner_output': 'sparse_categorical_crossentropy',
                    'pos_output': 'sparse_categorical_crossentropy' },

               metrics={
        'ner_output': 'accuracy',
        'pos_output': 'accuracy'}
    )

In [ ]:
#model fitting
model.fit(train_dataset, validation_data=valid_dataset, epochs=10)

In [ ]:
# Proqnozları al
pred_ner_probs, pred_pos_probs = model.predict(valid_dataset)

# Argmax ilə ən yüksək ehtimallı etiketləri seç
pred_ner_labels = np.argmax(pred_ner_probs, axis=-1)
pred_pos_labels = np.argmax(pred_pos_probs, axis=-1)

# İlk 2 nümunəni göstər
for i in range(2):
    print(f"\nExample {i+1}")
    print(f"{'Token':15} {'True NER':10} {'Pred NER':10} {'True POS':10} {'Pred POS':10}")
    print("-" * 60)

    input_ids = X_test[i]
    true_ner_ids = y_ner_test[i]
    pred_ner_ids = pred_ner_labels[i]
    true_pos_ids = y_pos_test[i]
    pred_pos_ids = pred_pos_labels[i]

    for token_id, true_ner_id, pred_ner_id, true_pos_id, pred_pos_id in zip(input_ids, true_ner_ids, pred_ner_ids, true_pos_ids, pred_pos_ids):
        if token_id != 0:  # padding tokenləri atla
            token = [k for k, v in vocab.items() if v == token_id][0]
            true_ner_tag = id2tag[true_ner_id]
            pred_ner_tag = id2tag[pred_ner_id]
            true_pos_tag = id2pos[true_pos_id]
            pred_pos_tag = id2pos[pred_pos_id]

            print(f"{token:15} {true_ner_tag:10} {pred_ner_tag:10} {true_pos_tag:10} {pred_pos_tag:10}")


In [ ]:
def predict_sentence(sentence, model, vocab, id2tag, id2pos, MAX_LEN):
    # 1. Tokenləşdir (sadə boşluqla ayırma)
    tokens = sentence.split()

    # 2. Tokenləri id-ə çevir
    token_ids = [vocab.get(token, 0) for token in tokens]

    # 3. Padding et
    input_ids = tf.keras.preprocessing.sequence.pad_sequences([token_ids], maxlen=MAX_LEN, padding='post', value=0)

    # 4. Pozisional id hazırla
    pos_ids = np.tile(np.arange(MAX_LEN), (1, 1))

    # 5. Modeldən proqnoz al
    pred_ner_probs, pred_pos_probs = model.predict([input_ids, pos_ids])

    # 6. Argmax et
    pred_ner = np.argmax(pred_ner_probs, axis=-1)[0]
    pred_pos = np.argmax(pred_pos_probs, axis=-1)[0]

    # 7. Nəticələri çap et
    print(f"{'Token':15} {'Predicted NER':15} {'Predicted POS'}")
    print("-" * 45)
    for i, token in enumerate(tokens):
        print(f"{token:15} {id2tag.get(pred_ner[i], 'O'):15} {id2pos.get(pred_pos[i], 'NN')}")

# Nümunə istifadə:
sentence = "Gloria Steinem works with the National Organization for Women to promote feminism."
predict_sentence(sentence, model, vocab, id2tag, id2pos, MAX_LEN)
